In [1]:
class SimpleDQ:
    import boto3
    glue_client = boto3.client('glue')

    def __init__(self, mode, **kwargs) -> None:
        mode = 'PyDeeQu', # ['PyDeeQu'|'GlueDQ']   
           

    def batch_get_data_quality_result(self, **kwargs): pass # ToDo 
    def cancel_data_quality_rule_recommendation_run(self, **kwargs): pass # ToDo 
    def cancel_data_quality_ruleset_evaluation_run(self, **kwargs): pass # ToDo

    def create_data_quality_ruleset(self, **kwargs): 

        


    def delete_data_quality_ruleset(self, **kwargs): pass # ToDo
    def get_data_quality_result(self, **kwargs): pass # ToDo
    def get_data_quality_rule_recommendation_run(self, **kwargs): pass # ToDo
    def get_data_quality_ruleset(self, **kwargs): pass # ToDo
    def get_data_quality_ruleset_evaluation_run(self, **kwargs): pass # ToDo
    def list_data_quality_results(self, **kwargs): pass # ToDo
    def list_data_quality_rule_recommendation_runs(self, **kwargs): pass # ToDo
    def list_data_quality_ruleset_evaluation_runs(self, **kwargs): pass # ToDo
    def list_data_quality_rulesets(self, **kwargs): pass # ToDo
    def start_data_quality_rule_recommendation_run(self, **kwargs): pass # ToDo
    def start_data_quality_ruleset_evaluation_run(self, **kwargs): pass # ToDo
    def update_data_quality_ruleset(self, **kwargs): pass # ToDo


In [6]:
def myfunc(**kwargs):
    for k,v in kwargs.items():
        print(k,v)

parms = {
    'Name' : 'name',
    'Description': 'desc',
    'Ruleset' : '[delimited string list]',
    'Tags' : {
        'string': 'string'
    },
    'TargetTable' : {
        'TableName': 'glueTable',
        'DatabaseName': 'glueDb',
        'CatalogId': 'default'
    },
    'ClientToken' :'uuid'
}

#myfunc(parms)

In [8]:
myfunc(**parms)

Name name
Description desc
Ruleset [delimited string list]
Tags {'string': 'string'}
TargetTable {'TableName': 'glueTable', 'DatabaseName': 'glueDb', 'CatalogId': 'default'}
ClientToken uuid
